# Jupyter Notebook Version of DL

The project is first developed in Jupyter Notebook for easy testing/verification but could be moved to a formal Python Script in the future (if I have time). Contrary to the Xtract-Sampler we won't be implementing any byte extraction but rather right now assume we have the data.
***
Training and Developing a model

### Import Statements

In [1]:
import pickle

### Loading Files/Data Processing

In [2]:
print("loading files now...")

with open('CDIACFileData/ByteVectors/byte_vector_dict_256KB_one_gram.pkl', "rb") as fp1:
    one_gram = pickle.load(fp1)
with open('CDIACFileData/ByteVectors/byte_vector_dict_256KB_two_gram.pkl', "rb") as fp2:
    two_gram = pickle.load(fp2)

print("loading files done!")

loading files now...


FileNotFoundError: [Errno 2] No such file or directory: 'CDIACFileData/ByteVectors/byte_vector_dict_256KB_one_gram.pkl'